In [0]:
from azure.storage.fileshare import ShareServiceClient
import os
import pydicom
from io import BytesIO
from pydicom.fileset import FileSet

In [0]:
acc_name = dbutils.secrets.get(scope = "adc_store", key = "pacs_intfileshare_accname")
acc_key = dbutils.secrets.get(scope = "adc_store", key = "pacs_intfileshare_acckey")

# Connection string
connection_string = f"DefaultEndpointsProtocol=https;AccountName={acc_name};AccountKey={acc_key};EndpointSuffix=core.windows.net"

# File share name
share_name = "intfileshare"

# Get a share client via connection string
share_client = ShareServiceClient.from_connection_string(connection_string).get_share_client(share_name)

# Source folder in the file share
source_folder = "sectra"

# List all files and directories in the root of the file share
print("Files and directories in the root of the file share:")
items = list(share_client.list_directories_and_files(source_folder))
for item in items:
    print(f"    {item['name']}")

In [0]:
file_client = share_client.get_file_client(f"{source_folder}/{items[1]['name']}/DICOMDIR")

In [0]:
file_bytes = file_client.download_file().readall()

In [0]:
with open("/Volumes/1_inland/sectra/vone/DICOMDIR", "wb") as f:
    f.write(file_bytes)

In [0]:
dcm_file = pydicom.dcmread("/Volumes/1_inland/sectra/vone/DICOMDIR")

In [0]:
list(dcm_file.DirectoryRecordSequence[3]["ReferencedFileID"].value)

In [0]:
rfids = []
for x in dcm_file.DirectoryRecordSequence:
    try:
        rfids.append(x["ReferencedFileID"].value)
    except:
        pass

In [0]:
rfids